# Building Agents with Tools in Python - Demo

This demo showcases how to build AI agents that can call external tools using OpenAI's function calling capabilities, applied to corporate financial planning scenarios.

## Learning Objectives
- Understand OpenAI function calling mechanics in a business context
- Learn to create tool schemas with proper validation
- Practice forcing specific tool execution with ```tool_choice```
- Implement error handling for tool calls
- Work with financial calculation tools

## Business Scenario
A financial services company needs AI-powered tools to help corporate clients with:
1. **ROI Calculator**: Calculate return on investment for corporate savings accounts
2. **Multi-Currency Converter**: Convert between major business currencies (USD, EUR, GBP, JPY)

In [1]:
# Environment Setup and Imports
import os, sys, json, argparse
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables and initialize OpenAI client for Vocareum
load_dotenv()
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")  # Load from .env file
)

# System prompt for the corporate financial assistant
SYSTEM = "You are a helpful corporate financial advisor. Provide clear, professional financial analysis and recommendations."

print("✅ Environment setup complete!")
print(f"✅ Using Vocareum OpenAI endpoint")
print(f"✅ API key loaded: {'YES' if os.getenv('OPENAI_API_KEY') else 'NO'}")

✅ Environment setup complete!
✅ Using Vocareum OpenAI endpoint
✅ API key loaded: YES


In [2]:
# Corporate Financial Tool Implementations

def calculate_roi(investment: float, profit: float, years: int) -> dict:
    """
    Calculate Return on Investment for corporate savings accounts.
    Returns dict with ROI percentage and annualized return.
    """
    if investment <= 0:
        raise ValueError("Investment amount must be positive")
    
    total_roi = (profit / investment) * 100
    annual_roi = (total_roi / years) if years > 0 else 0
    
    return {
        "initial_investment": round(investment, 2),
        "profit": round(profit, 2),
        "total_roi_percent": round(total_roi, 2),
        "annualized_roi_percent": round(annual_roi, 2),
        "investment_period_years": years
    }

def calculate_expense_deduction(gross_expenses: float, department: str, tax_rate: float = 0.21) -> dict:
    """
    Calculate corporate tax deductions for departmental expenses.
    Different departments have different deduction rates.
    Returns dict with expense breakdown and tax savings.
    """
    # Department-specific deduction rates (corporate tax rules)
    DEDUCTION_RATES = {
        "operations": 0.95,      # 95% deductible
        "marketing": 0.80,       # 80% deductible
        "research": 0.90,        # 90% deductible
        "facilities": 0.75,      # 75% deductible
        "training": 0.85         # 85% deductible
    }
    
    dept_lower = department.lower()
    if dept_lower not in DEDUCTION_RATES:
        raise ValueError(f"Unknown department: {department}. Supported: {', '.join(DEDUCTION_RATES.keys())}")
    
    deduction_rate = DEDUCTION_RATES[dept_lower]
    deductible_amount = gross_expenses * deduction_rate
    non_deductible_amount = gross_expenses - deductible_amount
    tax_savings = deductible_amount * tax_rate
    
    return {
        "gross_expenses": round(gross_expenses, 2),
        "department": department,
        "deduction_rate_percent": round(deduction_rate * 100, 1),
        "deductible_amount": round(deductible_amount, 2),
        "non_deductible_amount": round(non_deductible_amount, 2),
        "tax_rate_percent": round(tax_rate * 100, 1),
        "estimated_tax_savings": round(tax_savings, 2)
    }

print("✅ Corporate financial tool functions defined!")

✅ Corporate financial tool functions defined!


In [3]:
# OpenAI Tool Schemas

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "calculate_roi",
            "description": "Calculate Return on Investment for corporate investments and savings accounts.",
            "parameters": {
                "type": "object",
                "properties": {
                    "investment": {"type": "number", "description": "Initial investment amount in dollars"},
                    "profit": {"type": "number", "description": "Total profit earned in dollars"},
                    "years": {"type": "integer", "description": "Investment period in years"}
                },
                "required": ["investment", "profit", "years"],
                "additionalProperties": False
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_expense_deduction",
            "description": "Calculate corporate tax deductions for departmental expenses. Different departments have different deduction rates.",
            "parameters": {
                "type": "object",
                "properties": {
                    "gross_expenses": {"type": "number", "description": "Total gross departmental expenses in dollars"},
                    "department": {"type": "string", "enum": ["operations", "marketing", "research", "facilities", "training"], "description": "Department name"},
                    "tax_rate": {"type": "number", "description": "Corporate tax rate as decimal (default 0.21 for 21%)", "default": 0.21}
                },
                "required": ["gross_expenses", "department"],
                "additionalProperties": False
            },
        },
    },
]

# Map tool names to Python callables
FUNCTIONS = {
    "calculate_roi": calculate_roi,
    "calculate_expense_deduction": calculate_expense_deduction,
}

print("✅ Tool schemas defined!")

✅ Tool schemas defined!


In [4]:
# Core Tool Execution Logic

def agentic_tool_call(tool_name: str, user_content: str, tool_args: dict):
    """Direct tool call via tool_choice with agentic response generation."""
    # 1) Ask the model to call ONLY the specified tool (no ambiguity)
    first = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": SYSTEM},
                  {"role": "user", "content": user_content}],
        tools=TOOLS,
        tool_choice={"type": "function", "function": {"name": tool_name}},
        temperature=0.0,
    )
    msg = first.choices[0].message
    tool_call = msg.tool_calls[0] if msg.tool_calls else None

    # 2) Execute the Python function
    if tool_call is None:
        raise RuntimeError("Model did not issue the expected tool call.")
    name = tool_call.function.name
    model_args = json.loads(tool_call.function.arguments or "{}")
    call_args = {**model_args, **tool_args}
    result = FUNCTIONS[name](**call_args)

    # 3) Send tool result back and get final assistant reply
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": msg.content or "", "tool_calls": [tool_call.model_dump()]},
        {"role": "tool", "tool_call_id": tool_call.id, "name": name, "content": json.dumps(result)},
    ]
    final = client.chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=0.2)
    return final.choices[0].message.content

print("✅ Core execution logic defined!")

✅ Core execution logic defined!


In [5]:
# Test Functions for Interactive Use

def test_roi_calculator(investment, profit, years):
    """Test the ROI calculator with given parameters."""
    prompt = f"Our company invested ${investment:,} and earned ${profit:,} in profit over {years} years. Calculate our ROI. Provide a brief, clear analysis with the numbers. No formulas or LaTeX."
    response = agentic_tool_call(
        tool_name="calculate_roi",
        user_content=prompt,
        tool_args={"investment": investment, "profit": profit, "years": years},
    )
    return print(response)

def test_expense_calculator(gross_expenses, department, tax_rate=0.21):
    """Test the expense deduction calculator with given parameters."""
    prompt = f"Our {department} department had ${gross_expenses:,} in gross expenses. Calculate the deductible amounts and estimated tax savings at {tax_rate*100:.0f}% tax rate. Keep it brief and practical."
    response = agentic_tool_call(
        tool_name="calculate_expense_deduction",
        user_content=prompt,
        tool_args={"gross_expenses": gross_expenses, "department": department, "tax_rate": tax_rate},
    )
    print(response)

print("✅ Test functions ready!")

✅ Test functions ready!


In [6]:
# Demo: Corporate ROI Analysis

print("💼 Corporate Investment Analysis")
print("=" * 60)
print()

test_roi_calculator(investment=500000, profit=125000, years=3)

💼 Corporate Investment Analysis

Your company invested $500,000 and earned a profit of $125,000 over a period of 3 years. 

To analyze the return on investment (ROI):

1. **Total ROI**: The total ROI is 25%. This means that over the 3-year period, your investment has grown by a quarter of its original value.

2. **Annualized ROI**: The annualized ROI is approximately 8.33%. This figure represents the average return per year, indicating a steady growth rate over the investment period.

**Analysis**: A total ROI of 25% over 3 years is a positive outcome, reflecting a reasonable return on your investment. The annualized ROI of 8.33% is also competitive, especially in comparison to traditional savings or fixed-income investments. 

**Recommendation**: Consider evaluating other investment opportunities that may offer higher returns, while also assessing the associated risks. Additionally, it may be beneficial to reinvest profits to compound growth over time.
Your company invested $500,000 a

In [7]:
# Demo: Corporate Expense Deduction Analysis

print()
print("=" * 60)
print("Corporate Tax Deduction Analysis")
print("=" * 60)
print()

test_expense_calculator(gross_expenses=150000, department="research")


Corporate Tax Deduction Analysis

For your research department with gross expenses of $150,000, the deductible amount is $135,000, as 90% of the expenses are considered deductible. 

**Tax Calculation:**
- **Deductible Amount:** $135,000
- **Estimated Tax Savings:** $28,350 (calculated at a 21% tax rate)

**Summary:**
- **Total Gross Expenses:** $150,000
- **Deductible Amount:** $135,000
- **Estimated Tax Savings:** $28,350

This analysis suggests that maximizing your deductible expenses can significantly reduce your tax liability.
For your research department with gross expenses of $150,000, the deductible amount is $135,000, as 90% of the expenses are considered deductible. 

**Tax Calculation:**
- **Deductible Amount:** $135,000
- **Estimated Tax Savings:** $28,350 (calculated at a 21% tax rate)

**Summary:**
- **Total Gross Expenses:** $150,000
- **Deductible Amount:** $135,000
- **Estimated Tax Savings:** $28,350

This analysis suggests that maximizing your deductible expenses ca

# Demo Summary

## ✅ What We Demonstrated

**Technical Concepts:**
1. **OpenAI Function Calling**: Used `tool_choice` to force specific tool execution
2. **Complex Tool Schemas**: Defined schemas with enums and parameter validation
3. **Business Logic Integration**: Implemented corporate financial calculations
4. **Error Handling**: Added proper validation and error checking
5. **Multi-step AI Conversations**: Tool call → execution → AI analysis

**Business Applications:**
- Corporate investment ROI analysis
- Tax deduction calculations for departments